In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession 

sc = SparkContext()
spark = SparkSession.builder.appName("HW4-1").getOrCreate()

In [2]:
spotify = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/training_set/log_0_20180715_000000000000.csv")

In [3]:
feature = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/track_features")

## Merge Data

In [4]:
df = spotify.join(feature, spotify.track_id_clean == feature.track_id , "inner")
df.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- session_position: string (nullable = true)
 |-- session_length: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: string (nullable = true)
 |-- skip_2: string (nullable = true)
 |-- skip_3: string (nullable = true)
 |-- not_skipped: string (nullable = true)
 |-- context_switch: string (nullable = true)
 |-- no_pause_before_play: string (nullable = true)
 |-- short_pause_before_play: string (nullable = true)
 |-- long_pause_before_play: string (nullable = true)
 |-- hist_user_behavior_n_seekfwd: string (nullable = true)
 |-- hist_user_behavior_n_seekback: string (nullable = true)
 |-- hist_user_behavior_is_shuffle: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- premium: string (nullable = true)
 |-- context_type: string (nullable = true)
 |-- hist_user_behavior_reason_start: string (nullable = true)
 |-- hist_user_behavior_reason_end: string

In [6]:
df.count()

2990609

In [5]:
df.na.drop()

DataFrame[session_id: string, session_position: string, session_length: string, track_id_clean: string, skip_1: string, skip_2: string, skip_3: string, not_skipped: string, context_switch: string, no_pause_before_play: string, short_pause_before_play: string, long_pause_before_play: string, hist_user_behavior_n_seekfwd: string, hist_user_behavior_n_seekback: string, hist_user_behavior_is_shuffle: string, hour_of_day: string, date: string, premium: string, context_type: string, hist_user_behavior_reason_start: string, hist_user_behavior_reason_end: string, track_id: string, duration: string, release_year: string, us_popularity_estimate: string, acousticness: string, beat_strength: string, bounciness: string, danceability: string, dyn_range_mean: string, energy: string, flatness: string, instrumentalness: string, key: string, liveness: string, loudness: string, mechanism: string, mode: string, organism: string, speechiness: string, tempo: string, time_signature: string, valence: string, 

In [6]:
df.cache()

DataFrame[session_id: string, session_position: string, session_length: string, track_id_clean: string, skip_1: string, skip_2: string, skip_3: string, not_skipped: string, context_switch: string, no_pause_before_play: string, short_pause_before_play: string, long_pause_before_play: string, hist_user_behavior_n_seekfwd: string, hist_user_behavior_n_seekback: string, hist_user_behavior_is_shuffle: string, hour_of_day: string, date: string, premium: string, context_type: string, hist_user_behavior_reason_start: string, hist_user_behavior_reason_end: string, track_id: string, duration: string, release_year: string, us_popularity_estimate: string, acousticness: string, beat_strength: string, bounciness: string, danceability: string, dyn_range_mean: string, energy: string, flatness: string, instrumentalness: string, key: string, liveness: string, loudness: string, mechanism: string, mode: string, organism: string, speechiness: string, tempo: string, time_signature: string, valence: string, 

## Change Data Type

In [7]:
spotify.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- session_position: string (nullable = true)
 |-- session_length: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: string (nullable = true)
 |-- skip_2: string (nullable = true)
 |-- skip_3: string (nullable = true)
 |-- not_skipped: string (nullable = true)
 |-- context_switch: string (nullable = true)
 |-- no_pause_before_play: string (nullable = true)
 |-- short_pause_before_play: string (nullable = true)
 |-- long_pause_before_play: string (nullable = true)
 |-- hist_user_behavior_n_seekfwd: string (nullable = true)
 |-- hist_user_behavior_n_seekback: string (nullable = true)
 |-- hist_user_behavior_is_shuffle: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- premium: string (nullable = true)
 |-- context_type: string (nullable = true)
 |-- hist_user_behavior_reason_start: string (nullable = true)
 |-- hist_user_behavior_reason_end: string

In [8]:
columns_to_drop = ["hist_user_behavior_reason_start", "hist_user_behavior_reason_end", "context_type", "date"
                  ,"hour_of_day", "hist_user_behavior_is_shuffle", "hist_user_behavior_n_seekfwd", "session_length","session_position"]
df_drop = spotify.drop(*columns_to_drop)
df_drop.show(1)

+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|no_pause_before_play|short_pause_before_play|long_pause_before_play|hist_user_behavior_n_seekback|premium|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|             0|                   0|                      0|                     0|                            0|   true|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
only showi

In [9]:
from pyspark.sql.types import *

for c in ["skip_1", "skip_2", "skip_3", "not_skipped", "context_switch", "no_pause_before_play",
         "short_pause_before_play", "long_pause_before_play", "premium"]:
    df_drop = df_drop.withColumn(c, df_drop[c].cast(BooleanType()))
for c in ["hist_user_behavior_n_seekback"]:
    df_drop = df_drop.withColumn(c, df_drop[c].cast(IntegerType()))  
    
df_drop.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: boolean (nullable = true)
 |-- skip_2: boolean (nullable = true)
 |-- skip_3: boolean (nullable = true)
 |-- not_skipped: boolean (nullable = true)
 |-- context_switch: boolean (nullable = true)
 |-- no_pause_before_play: boolean (nullable = true)
 |-- short_pause_before_play: boolean (nullable = true)
 |-- long_pause_before_play: boolean (nullable = true)
 |-- hist_user_behavior_n_seekback: integer (nullable = true)
 |-- premium: boolean (nullable = true)



## Rating

In [10]:
df_drop.show(1)

+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|no_pause_before_play|short_pause_before_play|long_pause_before_play|hist_user_behavior_n_seekback|premium|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|         false|               false|                  false|                 false|                            0|   true|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
only showi

In [11]:
from pyspark.sql.functions import col

def pause_udf(short_pause, long_pause):
    if short_pause or long_pause:
        return 0
    else:
        return 1
    
pause = spark.udf.register("pause", pause_udf)

df1 = df_drop.select("session_id", "track_id_clean", "skip_1", "skip_2", "skip_3",
                     "not_skipped", "context_switch", "hist_user_behavior_n_seekback",
                     "premium", pause(col("short_pause_before_play"), col("long_pause_before_play")).alias("Not_Pause"))
df1.show(1)

#df_drop1 = df_drop.join(df1, ["session_id"] , "left")
df1 = df1.withColumn("Not_Pause", df1["Not_Pause"].cast(IntegerType()))
df1.printSchema()

+--------------------+--------------------+------+------+------+-----------+--------------+-----------------------------+-------+---------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|hist_user_behavior_n_seekback|premium|Not_Pause|
+--------------------+--------------------+------+------+------+-----------+--------------+-----------------------------+-------+---------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|         false|                            0|   true|        1|
+--------------------+--------------------+------+------+------+-----------+--------------+-----------------------------+-------+---------+
only showing top 1 row

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: boolean (nullable = true)
 |-- skip_2: boolean (nullable = true)
 |-- skip_3: boolean (nullable = true)
 |-- not_skipped: boolean (nullable = true)
 |-- context_switch: b

In [12]:
def skip_rating_udf(premium, skip_1, skip_2, skip_3, not_skip, nopause):
    if premium:
        if skip_1:
            skip1 = 1 - (1 - nopause)
        else:
            skip1 = 0
        if skip_2:
            skip2 = 3
        else:
            skip2 = 0
        if skip_3:
            skip3 = 5
        else:
            skip3 = 0
        if not_skip:
            notskip = 6 + (2 - 2 * nopause)
        else:
            notskip = 0
        return max(skip1, skip2, skip3, notskip)
    else:
        if skip_1:
            skip1 = 0
        else:
            skip1 = 0
        if skip_2:
            skip2 = 1
        else:
            skip2 = 0
        if skip_3:
            skip3 = 2
        else:
            skip3 = 0
        if not_skip:
            notskip = 5 + (1 - 1 * nopause)
        else:
            notskip = 0
        return max(skip1, skip2, skip3, notskip)
    
skip_rating = spark.udf.register("skip_rating", skip_rating_udf)

df2 = df1.select("session_id","context_switch","track_id_clean",
                 "hist_user_behavior_n_seekback",skip_rating(col("premium"), col("skip_1"), 
                                                             col("skip_2"), col("skip_3"), col("not_skipped"), 
                                                             col("Not_Pause")).alias("Skip_Rating"))
df2.show(5)

#df_drop2 = df_drop1.join(df2, ["session_id"] , "left")
df2 = df2.withColumn("Skip_Rating", df2["Skip_Rating"].cast(IntegerType()))

+--------------------+--------------+--------------------+-----------------------------+-----------+
|          session_id|context_switch|      track_id_clean|hist_user_behavior_n_seekback|Skip_Rating|
+--------------------+--------------+--------------------+-----------------------------+-----------+
|0_00006f66-33e5-4...|         false|t_0479f24c-27d2-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_9099cd7b-c238-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_fc5df5ba-5396-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_23cff8d6-d874-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_64f3743c-f624-4...|                            0|          6|
+--------------------+--------------+--------------------+-----------------------------+-----------+
only showing top 5 rows



In [13]:
def final_rating_udf(Skip_Rating, context_switch, seekback):
    if context_switch:
        context = 1
    else:
        context = 0
    inside = 1 - (Skip_Rating + 2*context)/10
    seekback = seekback + 1
    return (1 - pow(inside, (pow(seekback, 1/3)))) * 10

final_rating = spark.udf.register("final_rating", final_rating_udf)

df3 = df2.select("session_id", "track_id_clean",
                 final_rating(col("Skip_Rating"), col("context_switch"),
                              col("hist_user_behavior_n_seekback")).alias("Final_Rating"))
df3.show(5)

#df_drop3 = df_drop2.join(df3, ["session_id"] , "left")
df3 = df3.withColumn("Final_Rating", df3["Final_Rating"].cast(IntegerType()))

+--------------------+--------------------+------------+
|          session_id|      track_id_clean|Final_Rating|
+--------------------+--------------------+------------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...|         6.0|
|0_00006f66-33e5-4...|t_9099cd7b-c238-4...|         6.0|
|0_00006f66-33e5-4...|t_fc5df5ba-5396-4...|         6.0|
|0_00006f66-33e5-4...|t_23cff8d6-d874-4...|         6.0|
|0_00006f66-33e5-4...|t_64f3743c-f624-4...|         6.0|
+--------------------+--------------------+------------+
only showing top 5 rows



In [14]:
def adjust_udf(final):
    return final + 1

adjust = spark.udf.register("adjust", adjust_udf)

df4 = df3.select("session_id", "track_id_clean",
                 adjust(col("Final_Rating")).alias("Final_Rating_Adjust"))
df4.show(5)

#df_drop3 = df_drop2.join(df3, ["session_id"] , "left")
df4 = df4.withColumn("Final_Rating_Adjust", df4["Final_Rating_Adjust"].cast(IntegerType()))

+--------------------+--------------------+-------------------+
|          session_id|      track_id_clean|Final_Rating_Adjust|
+--------------------+--------------------+-------------------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...|                  7|
|0_00006f66-33e5-4...|t_9099cd7b-c238-4...|                  7|
|0_00006f66-33e5-4...|t_fc5df5ba-5396-4...|                  7|
|0_00006f66-33e5-4...|t_23cff8d6-d874-4...|                  7|
|0_00006f66-33e5-4...|t_64f3743c-f624-4...|                  7|
+--------------------+--------------------+-------------------+
only showing top 5 rows



In [28]:
df4.write.format("csv").option("header","true").mode("Overwrite").save("s3://olihw4/spotify_small_version2")

## Distribution 

In [16]:
df4.describe("Final_Rating_Adjust").show()

+-------+-------------------+
|summary|Final_Rating_Adjust|
+-------+-------------------+
|  count|            2990609|
|   mean|  5.857774118916916|
| stddev| 1.7515590800004013|
|    min|                  1|
|    max|                 11|
+-------+-------------------+



In [17]:
from pyspark.sql.functions import count
df4.groupBy("Final_Rating_Adjust").agg(count('session_id')).show()

+-------------------+-----------------+
|Final_Rating_Adjust|count(session_id)|
+-------------------+-----------------+
|                  1|            49933|
|                  6|          1657359|
|                  3|            13710|
|                  5|            11125|
|                  9|           124957|
|                  4|              201|
|                  8|            68940|
|                  7|           709088|
|                 10|             2326|
|                 11|             6413|
|                  2|           346557|
+-------------------+-----------------+



## Encoding

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [16]:
df4.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- Final_Rating_Adjust: integer (nullable = true)



In [17]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df4) 
            for column in ["session_id", "track_id_clean"] ]

pipeline = Pipeline(stages=indexers)
df_index = pipeline.fit(df4).transform(df4)

df_index.show()

+--------------------+--------------------+-------------------+----------------+--------------------+
|          session_id|      track_id_clean|Final_Rating_Adjust|session_id_index|track_id_clean_index|
+--------------------+--------------------+-------------------+----------------+--------------------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...|                  7|         48432.0|               309.0|
|0_00006f66-33e5-4...|t_9099cd7b-c238-4...|                  7|         48432.0|             13791.0|
|0_00006f66-33e5-4...|t_fc5df5ba-5396-4...|                  7|         48432.0|                34.0|
|0_00006f66-33e5-4...|t_23cff8d6-d874-4...|                  7|         48432.0|               735.0|
|0_00006f66-33e5-4...|t_64f3743c-f624-4...|                  7|         48432.0|               100.0|
|0_00006f66-33e5-4...|t_c815228b-3212-4...|                  6|         48432.0|               193.0|
|0_00006f66-33e5-4...|t_e23c19f5-4c32-4...|                  6|         48432.0|  

In [18]:
training, test = df_index.randomSplit([0.8, 0.2])

In [19]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="session_id_index", itemCol="track_id_clean_index", ratingCol="Final_Rating_Adjust",
          coldStartStrategy="drop")
model = als.fit(training)

In [20]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Final_Rating_Adjust",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 10.134819713933144


In [21]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

In [ ]:
userRecs.select("recommendations").show(10)

In [ ]:
userRecs.printSchema()

In [ ]:
movieRecs.show(5)

In [28]:
userRecs_df = userRecs.toPandas()

/usr/lib/spark/python/pyspark/sql/dataframe.py:2103: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: StructType(List(StructField(track_id_clean_index,IntegerType,true),StructField(rating,FloatType,true)))
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [29]:
userRecs_df.head()

,session_id_index,recommendations
0,148,"[(9399, 76.43775177001953), (33565, 74.4252166..."
1,463,"[(14135, 16.36673355102539), (38562, 16.025276..."
2,471,"[(18505, 53.014678955078125), (62867, 47.95021..."
3,496,"[(21956, 27.56434440612793), (62867, 26.973133..."
4,833,"[(10588, 48.45660400390625), (13720, 48.060390..."


In [30]:
movieRecs_df = movieRecs.toPandas()

/usr/lib/spark/python/pyspark/sql/dataframe.py:2103: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: StructType(List(StructField(session_id_index,IntegerType,true),StructField(rating,FloatType,true)))
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [31]:
movieRecs_df.head()

,track_id_clean_index,recommendations
0,148,"[(108204, 21.75996208190918), (119352, 20.3732..."
1,463,"[(51191, 54.46842575073242), (6149, 47.4529151..."
2,471,"[(122879, 51.11674118041992), (41636, 39.38167..."
3,496,"[(35636, 46.975887298583984), (109145, 44.6432..."
4,833,"[(107448, 43.743553161621094), (22702, 42.2848..."


In [32]:
df = df_index.toPandas()

In [40]:
df[df['session_id_index'] == 148]

,session_id,track_id_clean,Final_Rating_Adjust,session_id_index,track_id_clean_index
944002,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_2025e40b-aee8-45f4-80af-08c8a9752f26,6,148.0,2584.0
944003,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_de722614-f0e6-47af-81a6-64b3747e0cac,8,148.0,1793.0
944004,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_0fa66b51-205e-4fd2-9dfa-a775292e4a98,6,148.0,1605.0
944005,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_7c2e3cde-563e-446f-bdd5-cd696174643f,6,148.0,2924.0
944006,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_40b063c5-f805-4035-bcbc-08aab82860b0,7,148.0,1612.0
944007,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_7c2e3cde-563e-446f-bdd5-cd696174643f,6,148.0,2924.0
944008,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_0fa66b51-205e-4fd2-9dfa-a775292e4a98,6,148.0,1605.0
944009,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_7c2e3cde-563e-446f-bdd5-cd696174643f,6,148.0,2924.0
944010,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_40b063c5-f805-4035-bcbc-08aab82860b0,6,148.0,1612.0
944011,0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3,t_8233cb08-feeb-4c53-86fb-3d8ebb4c9aaa,6,148.0,6838.0


In [ ]:
0_50ae61e6-eca3-425f-ba4d-66dd0aee7df3

In [25]:
df_all = spotify.join(feature, spotify.track_id_clean == feature.track_id , "inner")

In [26]:
df_all.cache()

DataFrame[session_id: string, session_position: string, session_length: string, track_id_clean: string, skip_1: string, skip_2: string, skip_3: string, not_skipped: string, context_switch: string, no_pause_before_play: string, short_pause_before_play: string, long_pause_before_play: string, hist_user_behavior_n_seekfwd: string, hist_user_behavior_n_seekback: string, hist_user_behavior_is_shuffle: string, hour_of_day: string, date: string, premium: string, context_type: string, hist_user_behavior_reason_start: string, hist_user_behavior_reason_end: string, track_id: string, duration: string, release_year: string, us_popularity_estimate: string, acousticness: string, beat_strength: string, bounciness: string, danceability: string, dyn_range_mean: string, energy: string, flatness: string, instrumentalness: string, key: string, liveness: string, loudness: string, mechanism: string, mode: string, organism: string, speechiness: string, tempo: string, time_signature: string, valence: string, 

In [28]:
spotify = spotify.toPandas()

In [ ]:
t_2025e40b-aee8-45f4-80af-08c8a9752f26

In [39]:
spotify[spotify['session_id'] == 't_8233cb08-feeb-4c53-86fb-3d8ebb4c9aaa']

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end


In [32]:
feature.printSchema()

root
 |-- track_id: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- us_popularity_estimate: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- beat_strength: string (nullable = true)
 |-- bounciness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- dyn_range_mean: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- flatness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mechanism: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- organism: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- acoustic_vector_0: string (nullable = true)
 |-- acoustic_vector_1: 

In [40]:
newdf = feature.filter(feature['track_id']=='t_8233cb08-feeb-4c53-86fb-3d8ebb4c9aaa')

In [41]:
newdf.show()

+--------------------+------------------+------------+----------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+---+------------------+-------------------+-------------------+-----+-------------------+------------------+------------------+--------------+------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+
|            track_id|          duration|release_year|us_popularity_estimate|       acousticness|      beat_strength|         bounciness|       danceability|    dyn_range_mean|             energy|          flatness|    instrumentalness|key|          liveness|           loudness|          mechanism| mode|           organism|       speechiness|             tempo|time_signature|           valence|   acoustic_vector_0|  acoustic_vector_1| 

In [42]:
newdf = feature.filter(feature['track_id']=='t_0fa66b51-205e-4fd2-9dfa-a775292e4a98')

In [43]:
newdf.show()

+--------------------+------------------+------------+----------------------+-------------------+-------------------+-------------------+----------------+----------------+-------------------+------------------+--------------------+---+-------------------+------------------+------------------+-----+-------------------+--------------------+------------------+--------------+------------------+-----------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|            track_id|          duration|release_year|us_popularity_estimate|       acousticness|      beat_strength|         bounciness|    danceability|  dyn_range_mean|             energy|          flatness|    instrumentalness|key|           liveness|          loudness|         mechanism| mode|           organism|         speechiness|             tempo|time_signature|           valence|acoustic_vector_0|  acoustic_vector_1|  acoustic_vec

In [44]:
newdf = feature.filter(feature['track_id']=='t_7c2e3cde-563e-446f-bdd5-cd696174643f')

In [45]:
newdf.show()

+--------------------+------------------+------------+----------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+---+--------------------+-----------------+-------------------+-----+-----------------+--------------------+------------------+--------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            track_id|          duration|release_year|us_popularity_estimate|       acousticness|      beat_strength|         bounciness|       danceability|    dyn_range_mean|             energy|          flatness|    instrumentalness|key|            liveness|         loudness|          mechanism| mode|         organism|         speechiness|             tempo|time_signature|            valence|   acoustic_vector_0|  acoustic_vector_1|

In [46]:
newdf = feature.filter(feature['track_id']=='t_8233cb08-feeb-4c53-86fb-3d8ebb4c9aaa')

In [47]:
newdf.show()

+--------------------+------------------+------------+----------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+---+------------------+-------------------+-------------------+-----+-------------------+------------------+------------------+--------------+------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+
|            track_id|          duration|release_year|us_popularity_estimate|       acousticness|      beat_strength|         bounciness|       danceability|    dyn_range_mean|             energy|          flatness|    instrumentalness|key|          liveness|           loudness|          mechanism| mode|           organism|       speechiness|             tempo|time_signature|           valence|   acoustic_vector_0|  acoustic_vector_1| 